# Initialization

In [1]:
USE_MP = True
import silq
silq.initialize("EWJN")

Initializing imports


<IPython.core.display.Javascript object>

Initializing functions
Initializing instruments
Initializing layout
Initializing data
Initializing analysis
Initializing parameters
Initializing plotting
Initialization complete


In [2]:
slack = Slack()

In [3]:
qc.show_subprocess_widget()

Attempting to halt background measurement without termination
Attempting to halt background measurement without termination
Attempting to halt background measurement without termination
Attempting to halt background measurement without termination
Attempting to halt background measurement without termination


In [ ]:
calibration_operations = [
    {'mode': 'measure'},
    {'mode': '1D_scan', 'span': 0.007, 'set_points': 12, 'set_parameter': 'DF_DS'},
    {'mode': '1D_scan', 'span': 0.015, 'set_points': 25, 'set_parameter': 'DF_DS'}]
conditions =  [('contrast', 0.4, '>')]
autocal_parameter = measurement_parameters.CalibrationParameter(
    name='autocal', set_parameters=[DF_DS], acquisition_parameter=adiabatic_ESR_parameter, 
    operations=calibration_operations, discriminant='contrast', conditions=conditions)

## Threshold voltage

In [ ]:
EPR_parameter.single_settings(samples=50)
EPR_parameter.setup()
voltage_difference = EPR_parameter()[-1]

assert voltage_difference, "Couldn't find accurate threshold"
readout_threshold_voltage = voltage_difference / 2

print('Threshold voltage found at {:.2f} V'.format(readout_threshold_voltage))
properties_config['readout_threshold_voltage'] = readout_threshold_voltage

## ESR frequency

In [ ]:
select_ESR_parameter.single_settings(samples=30, silent=False)
select_ESR_parameter();

# DC Measurements

## DC - Turnon
Here several gates are combined to the parameter turnon_param.

In [ ]:
turnon_vals = turnon_parameter[list(np.linspace(1, 1.6, 100))]

In [ ]:
DC_parameter.setup()
data = qc.Loop(turnon_vals
# data = qc.Loop(dummy_parameter[0:1000:1], delay=1
               ).each(DC_parameter
                      ).then(qc.Task(layout.stop)
                             ).run(name='turnon',
                                   progress_interval=True)
plot = qc.MatPlot(data.DC_voltage)

## DC - Pinchoff
This code pinches off one gate, while keeping the other gates fixed

In [ ]:
turnon_voltage = 1.6
pinchoff_vals = list(np.linspace(turnon_voltage, 0, 100))
gate = TG

turnon_parameter(turnon_voltage)
DC_parameter.setup()
data = qc.Loop(gate[pinchoff_vals]
               ).each(DC_parameter
                      ).then(qc.Task(layout.stop)
                             ).run(name='Pinchoff_{}'.format(gate.name),
                                   progress_interval=True)
plot = qc.MatPlot(data.DC_voltage)

In [ ]:
turnon_voltage = 1.3
pinchoff_vals = list(np.linspace(turnon_voltage, 0.7, 40))
gatex=LB
gatey=RB

turnon_parameter(turnon_voltage)
DC_parameter.setup()
data = qc.Loop(gatey[pinchoff_vals]
               ).loop(gatex[pinchoff_vals]
                      ).each(DC_parameter
                      ).then(qc.Task(layout.stop)
                             ).run(name='2D_Pinchoff_{}'.format(gate.name),
                                   progress_interval=True)
plot = qc.MatPlot(data.DC_voltage)


## DC - 1D scan
This script performs a 1D DC scan

In [ ]:
gate = DF_DS[list(np.linspace(0.28,0.6, 50))]

DC_parameter.setup()
data = qc.Loop(gate
              ).each(DC_parameter
                    ).then(qc.Task(layout.stop)
                          ).run(name='DC_{}_scan'.format(gate.name),
                                progress_interval=True)

plot = qc.MatPlot(data.DC_voltage)

## DC - 2D scan

This measurement performs a 2D DC scan

In [83]:
y_sweep, x_sweep = create_set_vals(2, steps=15, points=20)
DC_parameter.setup()
data = qc.Loop(
    y_sweep).loop(
    x_sweep).each(
    DC_parameter).then(
    qc.Task(layout.stop)).run(
    name='DC_{}_vs_{}_scan'.format(y_sweep.name, x_sweep.name),
    progress_interval=True)

plot = qc.MatPlot(data.DC_voltage)

<silq.parameters.general_parameters.ScaledParameter: TGAC at 174650088>[0.197:0.2274:0.0016]
<silq.parameters.general_parameters.CombinedParameter: DF_DS at 188926160>[0.54009:0.55491:0.00078]
DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = '2016-12-25/#001_DC_TGAC_vs_DF_DS_scan_21-22-25'
   <Type>   | <array_id> | <array.name> | <array.shape>
   Setpoint | TGAC_set   | TGAC         | (20,)
   Setpoint | DF_DS_set  | DF_DS        | (20, 20)
   Measured | DC_voltage | DC_voltage   | (20, 20)
started at 2016-12-25 21:22:33


# EPR measurements (no steering/ESR)

This is a calibration routine to find a point with high contrast (difference in signal between up state and down state). 
This measurement consists of three stages: empty, load, and read.
In the empty stage, the donor atom is ionized. 
It is then loaded with an electron in a random state.
Finally the state is read out for a long duration.
The contrast is found by looking if blips occured at the beginning of the read trace and at the end of the read trace.
The contrast is the difference in percentage of blips occuring at the beginning versus at the end.
The end part of the read trace is related to the number of dark counts, since we assume that an up-state electron will have had enough time to have tunneled off and on again.

## EPR - 2D scan

In [ ]:
DF_DS(0.546)
TGAC(0.219)

In [ ]:
y_sweep, x_sweep = create_set_vals('2D', spans=0, nums=5)

In [ ]:
EPR_parameter.settings(samples=150)
data = qc.Loop(
    y_sweep).loop(
    x_sweep).each(
    EPR_parameter).run(
    name='EPR_calibration', progress_interval=True)

In [ ]:
plot = qc.MatPlot(subplots=(1, 3), figsize=(12, 4))
plot.add(data.contrast, subplot=1)
plot.add(data.dark_counts, subplot=2)
plot.add(data.voltage_difference, subplot=3)
plt.tight_layout()

## EPR - Single point

In [ ]:
EPR_parameter.single_settings(samples=150, silent=False)
EPR_parameter();

# Adiabatic sweep measurements

## ESR Adiabatic - Center frequency scan

This code sweeps over the adiabatic sweep center frequency to find the ESR frequency.

To find the ESR frequency, use 28GHz * B0 as starting point, and search around that frequency.
Use a big initial frequency_deviation (around 60 MHz) and long duration (0.5 ms)
Once you see a spike in contrast, center around there and decrease both deviation and duration.

In [22]:
adiabatic_ESR_parameter.pulse_sequence['adiabatic'].duration=0.15
adiabatic_ESR_parameter.pulse_sequence['adiabatic'].frequency_deviation = 1e6
adiabatic_ESR_parameter.pulse_sequence['adiabatic']

FrequencyRampPulse(adiabatic (ESR), f_center=27951.00 MHz, f_dev=1.00, power=0, t_start=3, t_stop=3.15)

In [58]:
f_center = properties_config['frequency_ESR']
f_span = 2e6
f_step = 0.1e6
adiabatic_sweep_vals = list(np.arange(f_center-f_span/2, f_center+f_span/2, f_step))
print('Sweep vals: {}'.format(len(adiabatic_sweep_vals)))

Sweep vals: 20


In [63]:
adiabatic_ESR_parameter.samples=40
data = qc.Loop(
    adiabatic_ESR_parameter[adiabatic_sweep_vals]).each(
    qc.Task(partial(print_attr, adiabatic_ESR_parameter, 'pulse_sequence')),
    adiabatic_ESR_parameter).run(
    name='adiabatic_ESR_f_center', progress_interval=True)
plot = qc.MatPlot(data.contrast, use_offset=False)

DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = '2016-12-24/#037_adiabatic_ESR_f_center_18-34-58'
   <Type>   | <array_id>         | <array.name>       | <array.shape>
   Setpoint | adiabatic_ESR_set  | adiabatic_ESR      | (20,)
   Measured | contrast           | contrast           | (20,)
   Measured | dark_counts        | dark_counts        | (20,)
   Measured | voltage_difference | voltage_difference | (20,)
started at 2016-12-24 18:35:00


## NMR adiabatic frequency scan

In [ ]:
#NMR sweep frequencies
f_center = 66.82e6
f_span = 3e6
f_step = 20e3
NMR_vals = list(np.arange(f_center-f_span/2, f_center+f_span/2, f_step))
print('Sweep vals: {}'.format(len(NMR_vals)))

In [ ]:
data = qc.Loop(adiabatic_NMR_parameter[NMR_vals]
              ).each(qc.Task(adiabatic_NMR_parameter.setup),
                     adiabatic_NMR_parameter,
                     qc.Task(partial(adiabatic_ESR_parameter, frequency_center_ESR['up'])),
                     adiabatic_ESR_parameter,
                     qc.Task(partial(adiabatic_ESR_parameter, frequency_center_ESR['down'])),
                     adiabatic_ESR_parameter,
                    ).run(name='NMR_adiabatic_sweep', progress_interval=True)

In [ ]:
plot = qc.MatPlot(data.contrast_3_4)
plot.add(data.contrast_5_4)

## Adiabatic - frequency deviation versus ESR duration

This is a 2D scan of frequency deviation versus duration.
From this measurement, the minimum acceptable values for frequency deviation and ESR duration can be extracted.

In [44]:
pulse = adiabatic_ESR_parameter.pulse_sequence['adiabatic']
adiabatic_deviation_parameter = general_parameters.AttributeParameter(object=pulse, attribute='frequency_deviation')
adiabatic_duration_parameter = general_parameters.AttributeParameter(object=pulse, attribute='duration')

adiabatic_deviation_vals = list(np.arange(0.5e6, 2.1e6, 0.25e6))
adiabatic_duration_vals = list(np.arange(0.075, 0.251, 0.025))

In [50]:
adiabatic_ESR_parameter.samples = 400
data = qc.Loop(
    adiabatic_deviation_parameter[adiabatic_deviation_vals]).loop(
    adiabatic_duration_parameter[adiabatic_duration_vals]).each(
    qc.Task(partial(print_attr, adiabatic_ESR_parameter, 'pulse_sequence')),
    adiabatic_ESR_parameter).run(
    name='adiabatic_deviation_vs_duration', progress_interval=True)

DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = '2016-12-24/#034_adiabatic_deviation_vs_duration_17-17-26'
   <Type>   | <array_id>              | <array.name>        | <array.shape>
   Setpoint | frequency_deviation_set | frequency_deviation | (7,)
   Setpoint | duration_set            | duration            | (7, 8)
   Measured | contrast                | contrast            | (7, 8)
   Measured | dark_counts             | dark_counts         | (7, 8)
   Measured | voltage_difference      | voltage_difference  | (7, 8)
started at 2016-12-24 17:17:28


In [51]:
plot = qc.MatPlot(subplots=(1,2), figsize=(12,5), interval=10)
plot.add(data.contrast, subplot=1, nticks=6)
plot.add(data.dark_counts, subplot=2, nticks=6)
# for ax in plot.subplots:
#     ax.set_xscale('log')
#     ax.set_yscale('log')
plot.fig.tight_layout(rect=[0, 0.03, 1, 0.95])

## Adiabatic - 2D scan

In [51]:
# TGAC, DF_DS sweep
y_sweep, x_sweep = create_set_vals(2, steps=0, points=7)

<silq.parameters.general_parameters.ScaledParameter: TGAC at 174330824>[0.4929:0.4959:0.0005]
<silq.parameters.general_parameters.CombinedParameter: DF_DS at 192904944>[0.41425:0.41574999999999995:0.00025]


In [52]:
adiabatic_ESR_parameter.samples = 200
data = qc.Loop(
    y_sweep).loop(
    x_sweep).each(
    adiabatic_ESR_parameter).run(
    name='adiabatic_calibration', progress_interval=True)

DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = '2016-12-29/#004_adiabatic_calibration_09-54-24'
   <Type>   | <array_id>         | <array.name>       | <array.shape>
   Setpoint | TGAC_set           | TGAC               | (7,)
   Setpoint | DF_DS_set          | DF_DS              | (7, 7)
   Measured | contrast           | contrast           | (7, 7)
   Measured | dark_counts        | dark_counts        | (7, 7)
   Measured | voltage_difference | voltage_difference | (7, 7)
started at 2016-12-29 09:54:27


In [53]:
CalibrationPlot(data)

## Adiabatic - 2D deviation versus duration

In [ ]:
pulse = adiabatic_ESR_parameter.pulse_sequence['adiabatic_sweep']
adiabatic_deviation_parameter = general_parameters.AttributeParameter(object=pulse, attribute='frequency_deviation')
adiabatic_duration_parameter = general_parameters.AttributeParameter(object=pulse, attribute='duration')


adiabatic_ESR_parameter.setup(samples=250, readout_threshold_voltage=readout_threshold_voltage,
                                save_traces=False)
data = qc.Loop(dummy_parameter[0:100:1]
              ).each(qc.Task(adiabatic_deviation_parameter, 5e6),
                     qc.Task(adiabatic_duration_parameter, .15),
                     qc.Task(adiabatic_ESR_parameter.setup),
                     adiabatic_ESR_parameter,
                     qc.Task(adiabatic_deviation_parameter, 15e6),
                     qc.Task(adiabatic_duration_parameter, .35),
                     qc.Task(adiabatic_ESR_parameter.setup),
                     adiabatic_ESR_parameter,
                    ).run(name='adiabatic_calibration',
                          progress_interval=True)

In [ ]:
plot = qc.MatPlot(data.contrast_3_4);
plot.add(data.contrast_7_4)

## Flip nucleus

In [ ]:
# Try to flip the nucleus
adiabatic_NMR_parameter(frequency_center_NMR)
adiabatic_NMR_parameter.samples = 1
adiabatic_NMR_parameter();

## Find ESR frequency

In [ ]:
select_ESR_parameter.setup(samples=50, print_results=True, readout_threshold_voltage=readout_threshold_voltage)
data = qc.Loop(dummy_parameter[0:100:1]).each(qc.Task(adiabatic_NMR_parameter.setup),
                                             adiabatic_NMR_parameter,
                                             qc.Task(select_ESR_parameter.setup),
                                             select_ESR_parameter,
                                             qc.Task(adiabatic_ESR_parameter, partial(getattr, select_ESR_parameter, 'frequency_ESR')),
                                             qc.Task(adiabatic_deviation_parameter, 5e6),
                                             qc.Task(adiabatic_duration_parameter, .15),
                                             qc.Task(adiabatic_ESR_parameter.setup),
                                             adiabatic_ESR_parameter,
                                             qc.Task(adiabatic_deviation_parameter, 15e6),
                                             qc.Task(adiabatic_duration_parameter, .35),
                                             qc.Task(adiabatic_ESR_parameter.setup),
                                             adiabatic_ESR_parameter,
                                            ).run(name='test_find_ESR',
                                                 progress_interval=True)

In [ ]:
plot = qc.MatPlot(data.contrast_up, label='nucleus up')
plot.add(data.contrast_down, label='nucleus down')
plot.add(data.contrast_8_4, label='ESR narrow')
plot.add(data.contrast_12_4, label='ESR wide')

# Dark counts

In [ ]:
dark_counts_parameter.single_settings(samples=150, silent=True)
dark_counts_parameter();

## Dark counts vs plunge voltage

In [56]:
plunge_voltage_parameter = general_parameters.ConfigPulseAttribute(
    pulse_name='plunge',
    attribute='amplitude')

In [63]:
dark_counts_plunge_parameter = acquisition_parameters.T1Parameter(mode='ESR')

dark_counts_plunge_parameter.pulse_sequence['adiabatic'].enabled = False
dark_counts_plunge_parameter.pulse_sequence['steered_initialization'].enabled = True
dark_counts_plunge_parameter.pulse_sequence['steered_initialization'].mode = 'long'
dark_counts_plunge_parameter.pulse_sequence['plunge'].duration = 20
dark_counts_plunge_parameter.pulse_sequence

PulseSequence with 5 pulses, duration: 42
	SteeredInitialization(steered_initialization (long), t_no_blip=200 ms, t_max_wait=1000, t_buffer=30, V_th=0.3333333333333333)
	DCPulse(plunge, A=2, t_start=0, t_stop=20)
	DCPulse(read, A=0, t_start=20, t_stop=40)
	DCPulse(final, A=0, t_start=40, t_stop=42)
	
	Disabled pulses:
	FrequencyRampPulse(adiabatic (ESR), f_center=27951.00 MHz, f_dev=0.50, power=0, t_start=3, t_stop=3.2)

In [76]:
dark_counts_plunge_parameter.samples=100
adiabatic_ESR_parameter.samples=100

T1_parameter(10)
data = qc.Loop(
    dummy_parameter[0:20:1]).loop(
    plunge_voltage_parameter[1:2.5:0.1]).each(
    qc.Task(partial(print_attr, dark_counts_plunge_parameter, 'pulse_sequence')),
    dark_counts_plunge_parameter,
    adiabatic_ESR_parameter).run(
    name='T1_plunge_voltage', progress_interval=True)

DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = '2016-12-29/#008_T1_plunge_voltage_11-08-08'
   <Type>   | <array_id>           | <array.name>       | <array.shape>
   Setpoint | dummy_set            | dummy              | (20,)
   Setpoint | plunge_amplitude_set | plunge_amplitude   | (20, 15)
   Measured | up_proportion        | up_proportion      | (20, 15)
   Measured | num_traces           | num_traces         | (20, 15)
   Measured | contrast             | contrast           | (20, 15)
   Measured | dark_counts          | dark_counts        | (20, 15)
   Measured | voltage_difference   | voltage_difference | (20, 15)
started at 2016-12-29 11:08:11


In [90]:
plot = qc.MatPlot(subplots=(1,3), figsize=(12,4))
plot.add(data.up_proportion, subplot=0)
plot.add(data.contrast, subplot=1)
plot.add(np.nanmean(data.contrast, 0), x=data.plunge_amplitude_set[0], subplot=2, label='contrast')
plot.add(np.nanmean(data.up_proportion, 0), x=data.plunge_amplitude_set[0], subplot=2, label='dark counts')
plot[2].set_xlabel('plunge_amplitude')
plot[2].legend()
plot.tight_layout()

# Read level sweep (to be fixed)

In [ ]:
variable_read_parameter.setup(samples=100)
data = qc.Loop(variable_read_parameter[-1.5:2.5:0.04]
                     ).each(variable_read_parameter
                           ).run(name='variable_read',
                                progress_interval=True)

In [ ]:
plotQ = qc.QtPlot()
plotQ.add(data.Channel_A_signal)

plotQ = qc.QtPlot()
plotQ.add(data.Channel_C_signal)

# T1 measurement

## T1 - Single point

In [ ]:
T1_parameter(20000)
T1_parameter.single_settings(samples=50)
%time up_proportion, num_traces = T1_parameter()

In [ ]:
up_proportion

In [ ]:
plot_traces(T1_parameter.trace_segments['output']['read'])

## T1 - single scan

In [42]:
# Calculate T1 durations
T1_wait_times = list(np.logspace(1, 3.5, num=10, base=10))

# Shuffle times
#np.random.shuffle(T1_wait_times)
print('Shuffled T1 wait times: {}'.format(T1_wait_times))

Shuffled T1 wait times: [10.0, 18.957356524063755, 35.938136638046274, 68.129206905796153, 129.15496650148839, 244.84367468222268, 464.15888336127819, 879.92254356910746, 1668.1005372000591, 3162.2776601683795]


In [43]:
# Single T1 sweep
T1_parameter.samples=50
data = qc.Loop(
    T1_parameter[T1_wait_times]).each(
    T1_parameter).run(name='T1_single_sweep')

plot = qc.MatPlot()
plot.add(data.up_proportion)
plot[0].set_xscale('log')

DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = '2016-12-27/#010_T1_single_sweep_12-37-44'
   <Type>   | <array_id>           | <array.name>     | <array.shape>
   Setpoint | T1_wait_time_ESR_set | T1_wait_time_ESR | (10,)
   Measured | up_proportion        | up_proportion    | (10,)
   Measured | num_traces           | num_traces       | (10,)
started at 2016-12-27 12:37:46


## T1 - Multiple scans

In [62]:
# Calculate T1 durations (shuffled)
T1_wait_times = list(np.logspace(1, 4.2, num=12, base=10))
np.random.shuffle(T1_wait_times)
print('Shuffled T1 wait times: {}'.format(T1_wait_times))

Shuffled T1 wait times: [1087.3356727196567, 74.598015234879369, 38.178440263705077, 8111.3083078968812, 19.539304046896113, 15848.931924611141, 10.0, 4151.2780027522995, 556.48638769832951, 284.80358684358049, 2124.5782310305694, 145.75933009692972]


In [111]:
T1_parameter.samples = 50
dark_counts_parameter.samples = 100
adiabatic_ESR_parameter.samples = 100
data = qc.Loop(
    dummy_parameter[0:80:1]).each(
    adiabatic_ESR_parameter,
    dark_counts_parameter,
    qc.Loop(T1_parameter[T1_wait_times]).each(T1_parameter),
                    ).run(name='T1_data', progress_interval=True)

DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = '2016-12-25/#009_T1_data_22-51-12'
   <Type>   | <array_id>           | <array.name>       | <array.shape>
   Setpoint | dummy_set            | dummy              | (80,)
   Measured | contrast             | contrast           | (80,)
   Measured | dark_counts_0_1      | dark_counts        | (80,)
   Measured | voltage_difference   | voltage_difference | (80,)
   Measured | dark_counts_1        | dark_counts        | (80,)
   Setpoint | T1_wait_time_ESR_set | T1_wait_time_ESR   | (80, 12)
   Measured | up_proportion        | up_proportion      | (80, 12)
   Measured | num_traces           | num_traces         | (80, 12)
started at 2016-12-25 22:51:14


In [112]:
plot = qc.MatPlot(subplots=(1,3), figsize=(12,3.5))
plot.add(data.up_proportion, subplot=1)
# plot.subplots[0].set_xscale('log')
# plot.add(data.T1_wait_time_set[0], np.nanmean(data.up_proportion_5_0_0, axis=0), subplot=2)
# plot.subplots[1].set_xscale('log')
# plot.subplots[1].set_xlim(10, 13000)
plot.add(data.contrast, subplot=3)
plot.add(data.dark_counts_1, subplot=3)
# plot.add(data.dark_counts_0_1, subplot=3)
plt.tight_layout()

## T1 - Plunge voltage scan
This scan measures the contrast for a fixed wait time as the plunge voltage is varied

In [11]:
plunge_voltage_parameter = general_parameters.ConfigPulseAttribute(
    pulse_name='plunge', attribute='amplitude')

In [49]:
T1_parameter.samples=40
adiabatic_ESR_parameter.samples=400
dark_counts_parameter.samples=250

T1_parameter(1500)
data = qc.Loop(
    dummy_parameter[0:70:1]).each(
    qc.Loop(plunge_voltage_parameter[1:2.5:0.1]).each(
        T1_parameter),
    adiabatic_ESR_parameter,
    dark_counts_parameter).run(
    name='T1_plunge_voltage', progress_interval=True)

DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = '2016-12-27/#011_T1_plunge_voltage_12-49-57'
   <Type>   | <array_id>         | <array.name>       | <array.shape>
   Setpoint | dummy_set          | dummy              | (70,)
   Setpoint | amplitude_set      | amplitude          | (70, 15)
   Measured | up_proportion      | up_proportion      | (70, 15)
   Measured | num_traces         | num_traces         | (70, 15)
   Measured | contrast           | contrast           | (70,)
   Measured | dark_counts_1_1    | dark_counts        | (70,)
   Measured | voltage_difference | voltage_difference | (70,)
   Measured | dark_counts_2      | dark_counts        | (70,)
started at 2016-12-27 12:49:59


In [59]:
plot = qc.MatPlot(subplots=(1,3), figsize=(12,4))
plot.add(data.up_proportion, subplot=1)
plot.add(np.nanmean(data.up_proportion, 0), subplot=2)
plot.add(data.contrast, subplot=3)
plot.add(data.dark_counts_2, subplot=3)
plt.tight_layout()

In [23]:
# Calculate T1 durations
T1_wait_times = list(np.logspace(1, 4, num=12, base=10))
print('T1 wait times: {}'.format(T1_wait_times))

T1 wait times: [10.0, 18.738174228603839, 35.111917342151308, 65.793322465756788, 123.28467394420659, 231.01297000831579, 432.87612810830569, 811.13083078968725, 1519.9110829529332, 2848.0358684357989, 5336.6992312063076, 10000.0]


In [24]:
dark_counts_plunge_parameter = acquisition_parameters.T1Parameter(mode='ESR')

dark_counts_plunge_parameter.pulse_sequence['adiabatic'].enabled = False
dark_counts_plunge_parameter.pulse_sequence['steered_initialization'].enabled = True
dark_counts_plunge_parameter.pulse_sequence['steered_initialization'].mode = 'long'
dark_counts_plunge_parameter.pulse_sequence['plunge'].duration = 20
dark_counts_plunge_parameter.pulse_sequence

In [26]:
T1_parameter.samples = 50
dark_counts_plunge_parameter.samples = 250
adiabatic_ESR_parameter.samples = 250

data = qc.Loop(
    dummy_parameter[0:30:1]).each(
    qc.Loop(
        plunge_voltage_parameter[1:2.5:0.1]).each(
        dark_counts_plunge_parameter,
        adiabatic_ESR_parameter),
    qc.Loop(
        T1_parameter[T1_wait_times]).loop(
            plunge_voltage_parameter[1:2.5:0.1]).each(
            T1_parameter)).run(
    name='T1_plunge_voltage', progress_interval=True)

DataSet:
   mode     = DataMode.PULL_FROM_SERVER
   location = '2016-12-27/#025_T1_plunge_voltage_21-53-45'
   <Type>   | <array_id>               | <array.name>       | <array.shape>
   Setpoint | dummy_set                | dummy              | (30,)
   Setpoint | plunge_amplitude_set_0   | plunge_amplitude   | (30, 15)
   Measured | up_proportion_0_0_0      | up_proportion      | (30, 15)
   Measured | num_traces_0_0_1         | num_traces         | (30, 15)
   Measured | contrast                 | contrast           | (30, 15)
   Measured | dark_counts              | dark_counts        | (30, 15)
   Measured | voltage_difference       | voltage_difference | (30, 15)
   Setpoint | T1_wait_time_ESR_set     | T1_wait_time_ESR   | (30, 12)
   Setpoint | plunge_amplitude_set_1_0 | plunge_amplitude   | (30, 12, 15)
   Measured | up_proportion_1_0_0_0    | up_proportion      | (30, 12, 15)
   Measured | num_traces_1_0_0_1       | num_traces         | (30, 12, 15)
started at 2016-12-27 21:5

In [31]:
plot = qc.MatPlot(subplots=(1, 2), figsize=(9, 4))
plot.add(data.contrast, subplot=0)
plot.add(data.dark_counts, subplot=1)

In [65]:
plot = qc.MatPlot()
plot.add(data.up_proportion)
plot.subplots[0].set_yscale('log')
plt.tight_layout()

# Notes

To save individual traces when performing a measurement, set:
    
EPR_parameter.setup(samples=100, data_manager=data_manager_raw)

In [ ]:
steered_initialization = adiabatic_sweep_parameter.pulse_sequence['steered_initialization']

In [ ]:
adiabatic_sweep_parameter.pulse_sequence['load'].t_stop=15

In [ ]:
adiabatic_sweep_parameter.steered_initialization

In [ ]:
steered_controller.record_initialization_traces

To enable/disable steered initialization, use:

adiabatic_sweep_parameter.steered_intialization = True/False

This adds/removes the steered initialization "pulse" from the EPR_parameter.pulse_sequence
If steered_initialization is enabled, you can access and modify its properties via

steered_initialization = adiabatic_sweep_parameter.pulse_sequence['steered_initialization']

Note that steered initialization is currently only implemented for the adiabatic sweep, but is easy to add for other parameters, such as T1_parameter (just add the pulse)

steered_controller is the acquisition controller specifically for steered initialization
You can look at the initialization traces when using steered initialization.

First set the flag 
steered_controller.record_initialization_traces(True)

The initialization traces can then be accessed
steered_controller.initialization_traces()

You can also look at the first buffer after the pulseblaster.start() command has been set via
steered_controller.post_initialization_traces()
This can be useful for checking the delay between sending the command and plunging

To setup a parameter where steered initialization is enabled, you must pass it a readout threshold voltage every time.

adiabatic_sweep_parameter.setup(readout_threshold_voltage=threshold_voltage)

This threshold voltage can be obtained from an EPR scan (see T1_measurement):

EPR_parameter.setup(samples=30)
EPR_parameter()
traces_read = EPR_parameter.trace_segments['output']['read']
_, _, readout_threshold_voltage = analysis.find_high_low(traces_read)

In [ ]:
adiabatic_sweep_parameter.pulse_sequence.add(adiabatic_sweep_parameter._steered_initialization)
adiabatic_sweep_parameter.pulse_sequence.remove(adiabatic_sweep_parameter._steered_initialization)

To save raw traces, use:

EPR_parameter.setup(save_traces=True, data_manager=data_manager_raw)

In [ ]:
plot = CalibrationPlot(data)

In [108]:
CalibrationPlot(dataset=data)